# 2.5. 自动微分

In [14]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [15]:
x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None

In [16]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [17]:
y.backward()
x.grad # 2 * x^2 的梯度为 4x，分别带入各x即为下结果

tensor([ 0.,  4.,  8., 12.])

In [19]:
x.grad == 4 * x # 验证

tensor([True, True, True, True])

In [23]:
# 在默认情况下，PyTorch会累积梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
print(y)
y.backward()
x.grad

tensor(6., grad_fn=<SumBackward0>)


tensor([1., 1., 1., 1.])

# 2.5.2. 非标量变量的反向传播

In [24]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 本例只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

# 2.5.3. 分离计算

In [25]:
x.grad.zero_()
y = x * x
u = y.detach()
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [26]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

# 2.5.4. Python控制流的梯度计算

In [27]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [28]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

In [29]:
a.grad == d / a

tensor(True)